In [18]:
"""
Unfold velocity and output file as cfradial
"""
import os
from glob import glob

import numpy as np
import pyart
import openradartools as ort
import vcor_dual_prf
import h5py

from matplotlib import pyplot as plt
import cartopy.crs as ccrs


In [4]:
source_folder = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset'
dest_folder = '/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset_dualprf'

In [32]:
odim_ffn_list = sorted(glob(source_folder + '/*.h5'))
for odim_ffn in odim_ffn_list:
    print(odim_ffn)
    radar = pyart.aux_io.read_odim_h5(odim_ffn)
    #insert dualprf data
    prt_mode_array = np.repeat(b'fixed', radar.nsweeps)
    prt_array = np.zeros(radar.nrays)
    prt_ratio_array = np.ones(radar.nrays)
    ny_array = np.zeros(radar.nrays)
    prf_flag_array = np.zeros(radar.nrays)
    #open file with h5py
    hfile = h5py.File(odim_ffn, 'r')

    for sw in range(0, radar.nsweeps):
        # start and end rays of sweep
        ray_s, ray_e = radar.get_start_end(sw)
        ray_e += 1
        # extract PRF/NI data from odimh5 file
        d_name = 'dataset' + str(sw+1)
        d_how = hfile[d_name]['how'].attrs
        # extract NI
        ny_array[ray_s:ray_e] = d_how['NI']
        # extract high PRF
        high_prf = d_how['highprf']
        prt_array[ray_s:ray_e] = 1/high_prf
        # check for dual prf
        low_prf = d_how['lowprf']
        if low_prf == 0:
            #Fixed PRF
            continue
        else:
            #dual prf
            prt_mode_array[sw] = b'dual'
            prt_ratio_array[ray_s:ray_e] = high_prf/low_prf
            mask = 0x0100
            flag_sw = prf_flag_array[ray_s:ray_e]
            if d_how['dataflag'][1] & mask == 0:
                #low prf first, 
                flag_sw[::2] = 1
            else:
                #high prf first,
                flag_sw[1::2] = 1
            prf_flag_array[ray_s:ray_e] = flag_sw

    # Create dictionaries
    mode_dict = {'comments': 'Pulsing mode Options are: "fixed", "staggered", "dual". Assumed "fixed" if missing.',
                'meta_group': 'instrument_parameters',
                'long_name': 'Pulsing mode',
                'units': 'unitless',
                'data': prt_mode_array}
    prt_dict = {'units': 'seconds',
                'comments': 'Pulse repetition time. For staggered prt, also see prt_ratio.',
                'meta_group': 'instrument_parameters',
                'long_name': 'Pulse repetition time',
                'data': prt_array}
    ratio_dict = {'units': 'unitless',
                'meta_group': 'instrument_parameters',
                'long_name': 'Pulse repetition frequency ratio',
                'data': prt_ratio_array}
    ny_dict = {'units': 'meters_per_second',
                'comments': 'Unambiguous velocity',
                'meta_group': 'instrument_parameters',
                'long_name': 'Nyquist velocity',
                'data': ny_array}
    flag_dict = {'units': 'unitless',
                'comments': 'PRF used to collect ray. 0 for high PRF, 1 for low PRF.',
                'meta_group': 'instrument_parameters',
                'long_name': 'PRF flag',
                'data': prf_flag_array}
    
    # add metadata in radar object:
    radar.instrument_parameters = {'nyquist_velocity':ny_dict, 'prt':prt_dict, 
                                'prt_ratio':ratio_dict, 'prt_mode':mode_dict, 
                                'prf_flag':flag_dict}
    
    
    #enforce masking
    vrad = radar.fields['velocity_horizontal']['data']
    vrad = np.ma.masked_equal(vrad, -999)
    vrad = np.ma.masked_invalid(vrad)
    radar.fields['velocity_horizontal']['data'] = vrad
    # run dualprf correction
    radar = vcor_dual_prf.correct_dualprf(radar=radar, two_step=True,
                                    method_det='cmean', kernel_det=np.ones((11, 11)),
                                    method_cor='cmean', kernel_cor=np.ones((5, 5)),
                                    vel_field='velocity_horizontal', new_field='velocity_dprf')
    
    #save to file
    out_ffn = f'{dest_folder}/{os.path.basename(odim_ffn)[:-3]}_dualprf.nc'
    pyart.io.write_cfradial(out_ffn, radar, format='NETCDF4')

    # #plot
    # min_lon=-116.35
    # max_lon=-115.85
    # min_lat=53.1
    # max_lat=53.80
    # sweep = 13
    # fig = plt.figure(figsize=(12, 5))
    # display = pyart.graph.RadarDisplay(radar)
    # ax = fig.add_subplot(121)
    # display.plot_ppi('velocity_horizontal', sweep=sweep, ax=ax, vmin=-30, vmax=20, cmap='BuDRd18')
    # ax.set_xlim(-170, -110)
    # ax.set_ylim(-45, 15)
    # ax = fig.add_subplot(122)
    # display.plot_ppi('velocity_dprf', sweep=sweep, ax=ax, vmin=-30, vmax=20, cmap='BuDRd18')
    # ax.set_xlim(-170, -110)
    # ax.set_ylim(-45, 15)





/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_24_ODIMH5_PVOL6S_VOL_CASCV.h5


/scratch/en0/jss548/miniforge3/envs/openradar/lib/python3.12/site-packages/vcor_dual_prf-1.0.0-py3.12.egg/vcor_dual_prf/vcor_dual_prf.py:262: RuntimeWarning: invalid value encountered in divide
  avg_ma = np.ma.array(data=sum_arr/valid_num, mask=mask)
/scratch/en0/jss548/miniforge3/envs/openradar/lib/python3.12/site-packages/pyart/io/cfradial.py:655: UserWarning: Unknown instrument parameter: prf_flag, not written to file.
  warnings.warn(message)


/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_30_ODIMH5_PVOL6S_VOL_CASCV.h5
/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_36_ODIMH5_PVOL6S_VOL_CASCV.h5
/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_42_ODIMH5_PVOL6S_VOL_CASCV.h5
/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_48_ODIMH5_PVOL6S_VOL_CASCV.h5
/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072422_54_ODIMH5_PVOL6S_VOL_CASCV.h5
/g/data/kl02/jss548/hail-research/hailsonde/20230724_CASCV_subset/2023072423_00_ODIMH5_PVOL6S_VOL_CASCV.h5
